# Annotate Cells
On this notebook, We annotate the cells by given **encoded annotation**.<br>
By using this, We can create **mask**.

In [ ]:
!pip install pycocotools

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pycocotools.mask as pyc
from PIL import Image
import cv2
plt.style.use('ggplot')

## Some Path

In [ ]:
train_path = Path("../input/sartorius-cell-instance-segmentation/train")
test_path = Path("../input/sartorius-cell-instance-segmentation/test")
meta_path = Path("../input/sartorius-cell-instance-segmentation/train.csv")

## Load meta-data

In [ ]:
meta_data = pd.read_csv(str(meta_path))
meta_data.head()

In [ ]:
labels = meta_data["cell_type"].drop_duplicates().tolist()
images_id = meta_data["id"].drop_duplicates().tolist()
print(f"# number of instances: {len(meta_data)}")
print(f"# number of images: {len(images_id)}")
print(f"# number of classes: {len(labels)}")

## Some function

In [ ]:
def rle_decode(ann):
    """
    decode run length encoded
    """
    t= ann
    ann = ann.split(" ")
    ans = []
    i = 0
    col = 0
    while i<len(ann)-1:
        pref = ann[i]
        l = ann[i+1]
        i+=2
        tm = []
        for j in range(int(l)):
            tm.append(int(pref)+j)
        ans+=tm
        col+=1
        
    return ans

def gen_color():
    b = np.random.randint(0,255)
    g = np.random.randint(0,255)
    r = np.random.randint(0,255)
    return r,g,b

In [ ]:
color = {
    "shsy5y": [0,255,0],
    "astro": [255,0,0],
    "cort": [0,0,255]
}

In [ ]:
plt.rcParams.update({'font.size': 60})
fig, axes = plt.subplots(2,2, figsize = (200,80),gridspec_kw={'height_ratios': [1, 1]})

sample_id = images_id[np.random.randint(0,len(images_id))]
instances = meta_data.loc[meta_data["id"] == sample_id]
h = instances["height"].drop_duplicates().tolist()[0]
w = instances["width"].drop_duplicates().tolist()[0]
im = cv2.imread(str(train_path.joinpath(f"{sample_id}.png")))
vec_im = im.copy().reshape((-1,im.shape[-1]))
bbox_im = im.copy().reshape((-1,im.shape[-1]))
global_mask = np.zeros((h*w,),dtype=np.uint8)
for idx in range(len(instances)):
    sample = instances.iloc[idx]
    ann = sample.annotation
    lb = sample.cell_type
    poses = rle_decode(ann)
    r,g,b = gen_color()
    mask = np.zeros((h*w,),dtype=np.uint8)
    for pos in poses:
        pos-=1
        
        vec_im[pos,0] = b
        vec_im[pos,1] = g
        vec_im[pos,2] = r
        
        bbox_im[pos,0] = b
        bbox_im[pos,1] = g
        bbox_im[pos,2] = r
        
        mask[pos] = 1
        global_mask[pos] = 1
    mask = mask.reshape((h,w))
    segmentation = pyc.encode(np.asarray(mask, order="F"))
    bbox=pyc.toBbox(segmentation)
    x0,y0,w0,h0 = bbox
    x0 = int(x0)
    y0 = int(y0)
    x1 = x0+int(w0)
    y1 = y0+int(h0)
    
    bbox_im = bbox_im.reshape(im.shape)
    bbox_im = cv2.rectangle(bbox_im,(x0,y0),(x1,y1),(b,g,r),2)
    bbox_im = bbox_im.reshape((-1,im.shape[-1]))
    
annotated_im = vec_im.reshape(im.shape)
bbox_im = bbox_im.reshape(im.shape)
global_mask = global_mask.reshape((h,w))

axes[0,0].imshow(im)
axes[0,0].grid(False)
axes[0,0].axis('off')
axes[0,0].set_title("Image")

axes[0,1].imshow(annotated_im)
axes[0,1].grid(False)
axes[0,1].axis('off')
axes[0,1].set_title("Annotation")

axes[1,0].imshow(bbox_im)
axes[1,0].grid(False)
axes[1,0].axis('off')
axes[1,0].set_title("Bounding Box")

axes[1,1].imshow(global_mask)
axes[1,1].grid(False)
axes[1,1].axis('off')
axes[1,1].set_title("Mask")

fig.show()